### cenaming
* https://github.com/portfoliome/cenaming
* Tools for removing organisation stop words. US Based. 

# name matching
* https://pypi.org/project/name-matching/

In [3]:
import os, sys, json
import numpy as np
import pandas as pd
import re

from name_matching.name_matcher import NameMatcher



Current working directory:  c:\Users\dec2g\GitHub\OrgSync


In [ ]:
def move_working_dir_to_repo_root(repo_name="orgsync"):
    current_dir = os.getcwd()
    #conver to lowercase
    while os.path.basename(current_dir).lower() != repo_name:
        current_dir = os.path.dirname(current_dir)
    os.chdir(current_dir)
    print("Current working directory: ", os.getcwd())

move_working_dir_to_repo_root(repo_name="orgsync")

In [30]:
data = "data/geolocations/orgs_names_postcodes.csv"
df = pd.read_csv(data)

df.head(100)

,dataset,id,name,short_name,postcode,city
0,cordis,323260,nuclear decommissioning authority - nda,nda,ca24 3hu,moor row
1,cordis,219438,south west tourism limited,south west tourism,ex2 5wt,exeter
2,cordis,219438,welsh government,welsh government,cf10 3nq,cardiff
3,cordis,226103,terrasalus limited,terrasalus,le15 9el,oakham
4,cordis,282558,ol pharma partners ltd,pharmivation,sg4 7dp,weston herts
...,...,...,...,...,...,...
95,cordis,321477,the technology strategy board,tsb,sn2 1jf,swindon
96,cordis,321477,biotechnology and biological sciences research...,bbsrc,sn2 1uh,swindon
97,cordis,321477,department for business energy and industrial ...,post brexit,sw1h 0et,london
98,cordis,312304,university of durham,university of durham,dh1 3le,durham


In [28]:
df_cordis = df[df['dataset']=="cordis"]
df_gtr = df[df['dataset']=="gtr"]


def keep_rows_in_col_with_any_keywords(df, col:str, keywords: str|list):
    if type(keywords) == str:
        return df[df[col].str.contains(keywords, case=False, na=False)]
    else:
        # if any of the keywords are in a row, keep it. If none are, drop it
        return df[df[col].str.contains('|'.join(keywords), case=False, na=False)]


def drop_rows_with_same_value_in_col(df, col:str):
    return df.drop_duplicates(subset=[col])

keywords = ["samsung", "apple", "microsoft"]
df_cordis = keep_rows_in_col_with_any_keywords(df_cordis, "name", keywords)
df_gtr = keep_rows_in_col_with_any_keywords(df_gtr, "name", keywords)

df_cordis = drop_rows_with_same_value_in_col(df_cordis, "name")
df_gtr = drop_rows_with_same_value_in_col(df_gtr, "name")

df_cordis.head(100)

,dataset,id,name,short_name,postcode,city
4317,cordis,216203,samsung electronics uk limited,samsung,kt16 0ps,chertsey - surrey
26715,cordis,765274,samsung healthcare,NaN,NaN,seúl


In [16]:
df_gtr.head(100)

,dataset,id,name,short_name,postcode,city
59469,gtr,473E5861-BBFC-4D27-8D56-85C8F23C7BCC,samsung ai centre (saic),NaN,cb1 2re,NaN
72312,gtr,7AFCB050-6ECC-4723-A0ED-BC1B7C4EC58F,samsung (south korea),NaN,unspecified,NaN
74946,gtr,BA06C5DE-A0B0-4FFA-AAAE-832228FE3B01,samsung electronics (uk) limited,NaN,kt16 0rs,NaN
97966,gtr,204D199E-EC60-4AD1-B546-A534863B17D7,samsung (united kingdom),NaN,tw18 4qe,NaN
99580,gtr,16D48577-2E41-4D9F-8822-A8E4AD22E365,samsung cambridge solution centre ltd,NaN,cb4 0ds,NaN
102793,gtr,21FEC5BD-16FC-4AC9-A7D2-F48F71FE2D61,samsung sds europe limited,NaN,kt13 0ny,NaN


In [20]:
# initialise the name matcher
matcher = NameMatcher(number_of_matches=10, 
                      legal_suffixes=True, 
                      common_words=True, 
                      top_n=50, 
                      verbose=True)

# adjust the distance metrics to use
matcher.set_distance_metrics(['bag', 'typo', 'refined_soundex'])

# load the data to which the names should be matched
matcher.load_and_process_master_data(column='name',
    df_matching_data=df_gtr, 
    transform=True)

# match names from the three datasets
cordis_matches = matcher.match_names(df_cordis, column_matching='name')
# cordis_matches_short = matcher.match(df_cordis_short, df_gtr)


cordis_matches.head()

preprocessing...

preprocessing complete 
 searching for matches...



100%|██████████| 1/1 [00:00<00:00, 999.12it/s]


possible matches found   
 fuzzy matching...



100%|██████████| 2/2 [00:00<00:00,  5.34it/s]

done


,original_name,match_name_0,score_0,match_index_0,match_name_1,score_1,match_index_1,match_name_2,score_2,match_index_2
4317,samsung electronics uk limited,samsung electronics uk limited,100.0,74946,samsung electronics uk limited,100.0,74946,samsung electronics uk limited,100.0,74946
26715,samsung healthcare,samsung south korea,0.0,72312,samsung south korea,0.0,72312,samsung south korea,0.0,72312


In [36]:
df.head(100)

,dataset,id,name,short_name,postcode,city
4317,cordis,216203,samsung electronics uk limited,samsung,kt16 0ps,chertsey - surrey
5692,cordis,222936,microsoft research limited,NaN,cb3 0fb,cambridge
25941,cordis,101002816,apples and snakes,apples and snakes,se8 4ag,london
26715,cordis,765274,samsung healthcare,NaN,NaN,seúl
43196,gtr,5EF90587-76D2-4D49-85B7-4423D997A090,tor lodge and applecross trust,NaN,b70 6py,NaN
43737,gtr,49EC9A45-34E2-4DD9-B267-01EBE3068B95,microsoft inria joint research centre,NaN,unspecified,NaN
47838,gtr,04454417-FE47-4A5C-9477-6C68BD48A10F,apple barn uk limited,NaN,ex20 4jx,NaN
48514,gtr,45708D35-EC8B-402A-963F-3E394C5C30A5,microsoft research,NaN,NaN,NaN
55449,gtr,56517A6E-4DA7-4D61-BD1D-3ACA5210D92E,microsoft corporation,NaN,NaN,NaN
56866,gtr,6E736620-DE1C-4204-A02C-57120C025E49,apple united states,NaN,95014,NaN


In [31]:
df = keep_rows_in_col_with_any_keywords(df, "name", ["samsung", "apple", "microsoft"])
df = drop_rows_with_same_value_in_col(df, "name")

df.head(100)

,dataset,id,name,short_name,postcode,city
4317,cordis,216203,samsung electronics (uk) limited,samsung,kt16 0ps,chertsey - surrey
5692,cordis,222936,microsoft research limited,NaN,cb3 0fb,cambridge
25941,cordis,101002816,apples and snakes,apples and snakes,se8 4ag,london
26715,cordis,765274,samsung healthcare,NaN,NaN,seúl
43196,gtr,5EF90587-76D2-4D49-85B7-4423D997A090,tor lodge and applecross trust,NaN,b70 6py,NaN
43737,gtr,49EC9A45-34E2-4DD9-B267-01EBE3068B95,microsoft inria joint research centre,NaN,unspecified,NaN
47838,gtr,04454417-FE47-4A5C-9477-6C68BD48A10F,apple barn uk limited,NaN,ex20 4jx,NaN
48514,gtr,45708D35-EC8B-402A-963F-3E394C5C30A5,microsoft research,NaN,NaN,NaN
55449,gtr,56517A6E-4DA7-4D61-BD1D-3ACA5210D92E,microsoft corporation,NaN,NaN,NaN
56866,gtr,6E736620-DE1C-4204-A02C-57120C025E49,apple (united states),NaN,95014,NaN


In [37]:


# initialise the name matcher
matcher = NameMatcher(number_of_matches=10, 
                      legal_suffixes=True, 
                      common_words=False, 
                      top_n=50, 
                      verbose=True)

# adjust the distance metrics to use
matcher.set_distance_metrics(['bag', 'typo', 'refined_soundex'])

# load the data to which the names should be matched
matcher.load_and_process_master_data(column='name',
    df_matching_data=df, 
    transform=True)

# match names from the three datasets
cordis_matches = matcher.match_names(df, column_matching='name')
# cordis_matches_short = matcher.match(df_cordis_short, df_gtr)


cordis_matches.head()

preprocessing...

preprocessing complete 
 searching for matches...



100%|██████████| 1/1 [00:00<00:00, 999.60it/s]


possible matches found   
 fuzzy matching...



100%|██████████| 33/33 [00:08<00:00,  3.90it/s]

done


,original_name,match_name_0,score_0,match_index_0,match_name_1,score_1,match_index_1,match_name_2,score_2,match_index_2,...,match_index_6,match_name_7,score_7,match_index_7,match_name_8,score_8,match_index_8,match_name_9,score_9,match_index_9
4317,samsung electronics uk limited,samsung electronics uk limited,100.0,4317,samsung electronics uk limited,100.000000,4317,samsung electronics uk limited,100.000000,4317,...,4317,samsung electronics uk limited,100.000000,4317,samsung electronics uk limited,100.000000,4317,samsung electronics uk limited,100.000000,4317
5692,microsoft research limited,microsoft research limited,100.0,5692,microsoft research united kingdom,100.000000,67429,microsoft research asia china,67.020336,100187,...,43737,microsoft united states,52.804233,64307,microsoft corporation,49.976292,55449,microsoft united kingdom,48.154962,105316
25941,apples and snakes,apples and snakes,100.0,25941,apple united states,60.878644,56866,apple united kingdom,51.101241,87462,...,69426,applegate marketplace limited,39.658878,100911,samsung ai centre saic,36.843368,59469,tor lodge and applecross trust,36.588341,43196
26715,samsung healthcare,samsung healthcare,100.0,26715,samsung south korea,70.857700,72312,samsung ai centre saic,61.562998,59469,...,4317,samsung electronics uk limited,60.707344,4317,samsung electronics uk limited,57.095141,4317,samsung electronics uk limited,46.422799,4317
43196,tor lodge and applecross trust,tor lodge and applecross trust,100.0,43196,harland and wolff appledore limited,44.448533,94447,rutherford appleton laboratory,44.263060,75551,...,56866,microsoft inria joint research centre,36.858379,43737,microsoft united states,36.588341,64307,apples and snakes,35.270738,25941
